In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# Import data from google drive
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
file_list = drive.ListFile({'q': "'1RxNKaI-lPF6L0jIx4hDimtsh1jNN3a3u' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: notMNIST.pickle, id: 1rymPKySp1y2D8QS-7-8WqG57k98wk0_Q


In [0]:
# Download req. pickle file
downloaded = drive.CreateFile({'id': '1rymPKySp1y2D8QS-7-8WqG57k98wk0_Q'})
downloaded.GetContentFile('notMNIST.pickle')

In [0]:
import pickle
import numpy as np

In [7]:
with open('notMNIST.pickle', 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [0]:
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD

## Baseline Model


In [10]:
# Set up the model
model = Sequential()

# Add the first layer
model.add(Dense(1024, activation = 'relu', input_shape = (784, )))

# Add the output layer
model.add(Dense(10, activation='softmax'))

# Create SGD optimizer with specified learning rate: my_optimizer
my_optimizer = SGD()
  

# Compile the model
model.compile(optimizer = my_optimizer, metrics = ['accuracy'], loss='categorical_crossentropy')

# Fit the model
with tf.device('/gpu:0'):
  model.fit(train_dataset, train_labels, epochs=10, batch_size=64, validation_data=(valid_dataset, valid_labels))

Train on 200000 samples, validate on 10000 samples
Epoch 1/10
200000/200000 [==============================] - 17s 83us/step - loss: 0.6667 - acc: 0.8177 - val_loss: 0.5408 - val_acc: 0.8450
Epoch 2/10
119680/200000 [================>.............] - ETA: 6s - loss: 0.5416 - acc: 0.8458

200000/200000 [==============================] - 17s 83us/step - loss: 0.5324 - acc: 0.8487 - val_loss: 0.4992 - val_acc: 0.8558
Epoch 3/10
200000/200000 [==============================] - 17s 83us/step - loss: 0.4957 - acc: 0.8581 - val_loss: 0.4725 - val_acc: 0.8651
Epoch 4/10
 31552/200000 [===>..........................] - ETA: 13s - loss: 0.4773 - acc: 0.8644

200000/200000 [==============================] - 17s 83us/step - loss: 0.4692 - acc: 0.8657 - val_loss: 0.4535 - val_acc: 0.8677
Epoch 5/10
200000/200000 [==============================] - 17s 83us/step - loss: 0.4475 - acc: 0.8716 - val_loss: 0.4367 - val_acc: 0.8723
Epoch 6/10
 11328/200000 [>.............................] - ETA: 15s - loss: 0.4358 - acc: 0.8754

200000/200000 [==============================] - 17s 83us/step - loss: 0.4291 - acc: 0.8770 - val_loss: 0.4235 - val_acc: 0.8767
Epoch 7/10
200000/200000 [==============================] - 17s 83us/step - loss: 0.4131 - acc: 0.8818 - val_loss: 0.4115 - val_acc: 0.8801
Epoch 8/10
  5888/200000 [..............................] - ETA: 15s - loss: 0.4067 - acc: 0.8838

200000/200000 [==============================] - 17s 83us/step - loss: 0.3992 - acc: 0.8854 - val_loss: 0.4028 - val_acc: 0.8841
Epoch 9/10
200000/200000 [==============================] - 17s 83us/step - loss: 0.3867 - acc: 0.8890 - val_loss: 0.3941 - val_acc: 0.8855
Epoch 10/10
  4672/200000 [..............................] - ETA: 16s - loss: 0.3939 - acc: 0.8851

200000/200000 [==============================] - 17s 83us/step - loss: 0.3753 - acc: 0.8920 - val_loss: 0.3883 - val_acc: 0.8887


In [13]:
# Score trained model.
scores = model.evaluate(test_dataset, test_labels, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 58us/step
Test loss: 0.21211404593586922
Test accuracy: 0.9418


## L2 REGULARIZATION

In [0]:
from keras import regularizers

In [41]:
# Set up the model
model = Sequential()

# Add the first layer
model.add(Dense(1024, activation = 'relu', input_shape = (784, ),kernel_regularizer=regularizers.l2(.00005),activity_regularizer=regularizers.l2(.00005), bias_regularizer = regularizers.l2(.00005)))

# Add the output layer
model.add(Dense(10, activation='softmax'))

# Create SGD optimizer with specified learning rate: my_optimizer
my_optimizer = SGD()
  

# Compile the model
model.compile(optimizer = my_optimizer, metrics = ['accuracy'], loss='categorical_crossentropy')

# Fit the model
with tf.device('/gpu:0'):
  model.fit(train_dataset, train_labels, epochs=50, batch_size=64, validation_data=(valid_dataset, valid_labels))

Train on 200000 samples, validate on 10000 samples
Epoch 1/50
200000/200000 [==============================] - 19s 97us/step - loss: 0.9346 - acc: 0.8165 - val_loss: 0.7601 - val_acc: 0.8428
Epoch 2/50
 92736/200000 [============>.................] - ETA: 10s - loss: 0.7531 - acc: 0.8429

200000/200000 [==============================] - 19s 94us/step - loss: 0.7332 - acc: 0.8458 - val_loss: 0.6846 - val_acc: 0.8534
Epoch 3/50
200000/200000 [==============================] - 19s 95us/step - loss: 0.6725 - acc: 0.8554 - val_loss: 0.6420 - val_acc: 0.8605
Epoch 4/50
  5184/200000 [..............................] - ETA: 18s - loss: 0.6328 - acc: 0.8605

200000/200000 [==============================] - 19s 95us/step - loss: 0.6315 - acc: 0.8629 - val_loss: 0.6102 - val_acc: 0.8669
Epoch 5/50
179328/200000 [=========================>....] - ETA: 1s - loss: 0.6018 - acc: 0.8690

200000/200000 [==============================] - 19s 93us/step - loss: 0.5997 - acc: 0.8693 - val_loss: 0.5856 - val_acc: 0.8708
Epoch 6/50
200000/200000 [==============================] - 19s 94us/step - loss: 0.5734 - acc: 0.8746 - val_loss: 0.5660 - val_acc: 0.8754
Epoch 7/50
 26816/200000 [===>..........................] - ETA: 15s - loss: 0.5535 - acc: 0.8789

200000/200000 [==============================] - 19s 94us/step - loss: 0.5515 - acc: 0.8794 - val_loss: 0.5491 - val_acc: 0.8786
Epoch 8/50
187072/200000 [===========================>..] - ETA: 1s - loss: 0.5325 - acc: 0.8838

...

200000/200000 [==============================] - 19s 94us/step - loss: 0.2907 - acc: 0.9475 - val_loss: 0.4102 - val_acc: 0.9086
Epoch 45/50
200000/200000 [==============================] - 19s 94us/step - loss: 0.2872 - acc: 0.9484 - val_loss: 0.4102 - val_acc: 0.9094
Epoch 46/50
 26944/200000 [===>..........................] - ETA: 16s - loss: 0.2783 - acc: 0.9519

200000/200000 [==============================] - 19s 94us/step - loss: 0.2841 - acc: 0.9495 - val_loss: 0.4080 - val_acc: 0.9088
Epoch 47/50
184064/200000 [==========================>...] - ETA: 1s - loss: 0.2806 - acc: 0.9505

200000/200000 [==============================] - 19s 94us/step - loss: 0.2807 - acc: 0.9504 - val_loss: 0.4076 - val_acc: 0.9095
Epoch 48/50
200000/200000 [==============================] - 19s 95us/step - loss: 0.2776 - acc: 0.9515 - val_loss: 0.4075 - val_acc: 0.9092
Epoch 49/50
 25856/200000 [==>...........................] - ETA: 16s - loss: 0.2696 - acc: 0.9546

200000/200000 [==============================] - 19s 94us/step - loss: 0.2746 - acc: 0.9522 - val_loss: 0.4091 - val_acc: 0.9092
Epoch 50/50
184576/200000 [==========================>...] - ETA: 1s - loss: 0.2707 - acc: 0.9538

200000/200000 [==============================] - 19s 94us/step - loss: 0.2715 - acc: 0.9534 - val_loss: 0.4054 - val_acc: 0.9098


In [43]:
# Score trained model.
scores = model.evaluate(test_dataset, test_labels, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 74us/step
Test loss: 0.21098018407821656
Test accuracy: 0.9586


## Adding one dropout layer

In [48]:
# Set up the model
model = Sequential()

# Add the first layer and DROPOUT 
model.add(Dense(1024, activation = 'relu', input_shape = (784, ),kernel_regularizer=regularizers.l2(.00005),activity_regularizer=regularizers.l2(.00005), bias_regularizer = regularizers.l2(.00005)))
model.add(Dropout(0.25))

# Add the output layer
model.add(Dense(10, activation='softmax'))

# Create SGD optimizer with specified learning rate: my_optimizer
my_optimizer = SGD()
  

# Compile the model
model.compile(optimizer = my_optimizer, metrics = ['accuracy'], loss='categorical_crossentropy')

# Fit the model
with tf.device('/gpu:0'):
  model.fit(train_dataset, train_labels, epochs=50, batch_size=64, validation_data=(valid_dataset, valid_labels))

Train on 200000 samples, validate on 10000 samples
Epoch 1/50
200000/200000 [==============================] - 20s 102us/step - loss: 0.9477 - acc: 0.8106 - val_loss: 0.7606 - val_acc: 0.8427
Epoch 2/50
 85248/200000 [===========>..................] - ETA: 11s - loss: 0.7654 - acc: 0.8396

200000/200000 [==============================] - 20s 100us/step - loss: 0.7440 - acc: 0.8425 - val_loss: 0.6854 - val_acc: 0.8538
Epoch 3/50
193152/200000 [===========================>..] - ETA: 0s - loss: 0.6849 - acc: 0.8521

200000/200000 [==============================] - 20s 100us/step - loss: 0.6851 - acc: 0.8519 - val_loss: 0.6443 - val_acc: 0.8594
Epoch 4/50
200000/200000 [==============================] - 20s 100us/step - loss: 0.6449 - acc: 0.8588 - val_loss: 0.6135 - val_acc: 0.8635
Epoch 5/50
 21824/200000 [==>...........................] - ETA: 17s - loss: 0.6139 - acc: 0.8678

200000/200000 [==============================] - 20s 100us/step - loss: 0.6144 - acc: 0.8645 - val_loss: 0.5896 - val_acc: 0.8686
Epoch 6/50
171200/200000 [========================>.....] - ETA: 2s - loss: 0.5922 - acc: 0.8685

200000/200000 [==============================] - 20s 100us/step - loss: 0.5901 - acc: 0.8692 - val_loss: 0.5702 - val_acc: 0.8722
Epoch 7/50
200000/200000 [==============================] - 20s 100us/step - loss: 0.5683 - acc: 0.8739 - val_loss: 0.5549 - val_acc: 0.8748
Epoch 8/50
 15616/200000 [=>............................] - ETA: 18s - loss: 0.5667 - acc: 0.8730

...

200000/200000 [==============================] - 20s 100us/step - loss: 0.3213 - acc: 0.9336 - val_loss: 0.4082 - val_acc: 0.9068
Epoch 45/50
167360/200000 [========================>.....] - ETA: 3s - loss: 0.3190 - acc: 0.9345

200000/200000 [==============================] - 20s 100us/step - loss: 0.3183 - acc: 0.9345 - val_loss: 0.4073 - val_acc: 0.9080
Epoch 46/50
200000/200000 [==============================] - 20s 100us/step - loss: 0.3151 - acc: 0.9352 - val_loss: 0.4077 - val_acc: 0.9090
Epoch 47/50
 14592/200000 [=>............................] - ETA: 18s - loss: 0.3058 - acc: 0.9381

200000/200000 [==============================] - 20s 100us/step - loss: 0.3126 - acc: 0.9359 - val_loss: 0.4049 - val_acc: 0.9082
Epoch 48/50
167744/200000 [========================>.....] - ETA: 3s - loss: 0.3085 - acc: 0.9373

200000/200000 [==============================] - 20s 99us/step - loss: 0.3097 - acc: 0.9369 - val_loss: 0.4051 - val_acc: 0.9083
Epoch 49/50
200000/200000 [==============================] - 20s 99us/step - loss: 0.3069 - acc: 0.9377 - val_loss: 0.4051 - val_acc: 0.9095
Epoch 50/50
 17280/200000 [=>............................] - ETA: 17s - loss: 0.3038 - acc: 0.9387

200000/200000 [==============================] - 20s 99us/step - loss: 0.3040 - acc: 0.9378 - val_loss: 0.4030 - val_acc: 0.9099


In [49]:
# Score trained model.
scores = model.evaluate(test_dataset, test_labels, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 69us/step
Test loss: 0.2142465713262558
Test accuracy: 0.9569


## Dropout Layer without regularization

In [50]:
# Set up the model
model = Sequential()

# Add the first layer AND DROPOUT 
model.add(Dense(1024, activation = 'relu', input_shape = (784, )))
model.add(Dropout(0.25))

# Add the output layer
model.add(Dense(10, activation='softmax'))

# Create SGD optimizer with specified learning rate: my_optimizer
my_optimizer = SGD()
  

# Compile the model
model.compile(optimizer = my_optimizer, metrics = ['accuracy'], loss='categorical_crossentropy')

# Fit the model
with tf.device('/gpu:0'):
  model.fit(train_dataset, train_labels, epochs=50, batch_size=64, validation_data=(valid_dataset, valid_labels))

Train on 200000 samples, validate on 10000 samples
Epoch 1/50
200000/200000 [==============================] - 18s 92us/step - loss: 0.6862 - acc: 0.8094 - val_loss: 0.5410 - val_acc: 0.8453
Epoch 2/50
102912/200000 [==============>...............] - ETA: 8s - loss: 0.5614 - acc: 0.8400

200000/200000 [==============================] - 18s 91us/step - loss: 0.5502 - acc: 0.8429 - val_loss: 0.5002 - val_acc: 0.8569
Epoch 3/50
200000/200000 [==============================] - 18s 91us/step - loss: 0.5140 - acc: 0.8515 - val_loss: 0.4756 - val_acc: 0.8627
Epoch 4/50
 11584/200000 [>.............................] - ETA: 17s - loss: 0.4812 - acc: 0.8639

200000/200000 [==============================] - 18s 91us/step - loss: 0.4888 - acc: 0.8587 - val_loss: 0.4573 - val_acc: 0.8694
Epoch 5/50
186816/200000 [===========================>..] - ETA: 1s - loss: 0.4703 - acc: 0.8637

200000/200000 [==============================] - 18s 91us/step - loss: 0.4694 - acc: 0.8639 - val_loss: 0.4408 - val_acc: 0.8727
Epoch 6/50
200000/200000 [==============================] - 18s 92us/step - loss: 0.4526 - acc: 0.8688 - val_loss: 0.4286 - val_acc: 0.8762
Epoch 7/50
 31808/200000 [===>..........................] - ETA: 15s - loss: 0.4399 - acc: 0.8718

200000/200000 [==============================] - 18s 91us/step - loss: 0.4381 - acc: 0.8730 - val_loss: 0.4181 - val_acc: 0.8798
Epoch 8/50
195008/200000 [============================>.] - ETA: 0s - loss: 0.4252 - acc: 0.8772

...

200000/200000 [==============================] - 18s 91us/step - loss: 0.2638 - acc: 0.9234 - val_loss: 0.3245 - val_acc: 0.9070
Epoch 45/50
200000/200000 [==============================] - 18s 91us/step - loss: 0.2609 - acc: 0.9241 - val_loss: 0.3232 - val_acc: 0.9089
Epoch 46/50
 33856/200000 [====>.........................] - ETA: 14s - loss: 0.2575 - acc: 0.9247

200000/200000 [==============================] - 18s 91us/step - loss: 0.2577 - acc: 0.9254 - val_loss: 0.3233 - val_acc: 0.9091
Epoch 47/50
194496/200000 [============================>.] - ETA: 0s - loss: 0.2572 - acc: 0.9253

200000/200000 [==============================] - 18s 91us/step - loss: 0.2570 - acc: 0.9254 - val_loss: 0.3242 - val_acc: 0.9088
Epoch 48/50
200000/200000 [==============================] - 18s 91us/step - loss: 0.2537 - acc: 0.9267 - val_loss: 0.3228 - val_acc: 0.9094
Epoch 49/50
 33728/200000 [====>.........................] - ETA: 14s - loss: 0.2512 - acc: 0.9267

200000/200000 [==============================] - 18s 91us/step - loss: 0.2517 - acc: 0.9268 - val_loss: 0.3217 - val_acc: 0.9095
Epoch 50/50
194880/200000 [============================>.] - ETA: 0s - loss: 0.2497 - acc: 0.9273

200000/200000 [==============================] - 18s 91us/step - loss: 0.2492 - acc: 0.9273 - val_loss: 0.3224 - val_acc: 0.9092


In [53]:
# Score trained model.
scores = model.evaluate(test_dataset, test_labels, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 65us/step
Test loss: 0.14998465966880323
Test accuracy: 0.9592
